In [13]:
# 載入需要的套件
import os
import numpy as np 
import pandas as pd
import copy
import seaborn as sns
import xgboost as xgb
from scipy.stats import skew
from lightgbm import LGBMRegressor
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler, StandardScaler, Imputer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
%matplotlib inline

In [274]:
# 設定 data_path
dir_data = './data/'
Train = os.path.join(dir_data, 'train.csv')
Test = os.path.join(dir_data, 'test.csv')

# 讀取檔案
Train_data = pd.read_csv(Train)
Test_data = pd.read_csv(Test)

In [275]:
ids = Test_data['building_id']

Train_data = Train_data.drop(['building_id'] , axis=1)
Test_data = Test_data.drop(['building_id'] , axis=1)

In [144]:
# 檢查 DataFrame 空缺值的狀態
def na_check(df_data):
    data_na = (df_data.isnull().sum() / len(df_data)) * 100
    data_na = data_na.drop(data_na[data_na == 0].index).sort_values(ascending=False)
    missing_data = pd.DataFrame({'Missing Ratio' :data_na})
    display(missing_data.head(10))
    
def na_check2(df_data):
    #missing data
    total = df_data.isnull().sum().sort_values(ascending=False)
    percent = (df_data.isnull().sum()/df_data.isnull().count()).sort_values(ascending=False)
    missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    display(missing_data.head(4))
    
def area_type(row):
    if row >= 0:
        return 1
    else:
        return 0
    
def house_type(row):
    if row == 0:
        return 0
    else:
        return 1
    
def lat_diff(row):
    if row > -38.6:
        return 1
    else:
        return 0
    
def Parking_area_Fill(data):
    if np.isnan(data['parking_area']):
        if not np.isnan(data['Parking_area_Filling']):
            return data['Parking_area_Filling']
    return data['parking_area']

In [276]:
Train_data[Train_data.loc[:,"total_price"] > 3000000000]["total_floor"]
Train_data = Train_data.drop(2138)
Train_data = Train_data.drop(2317)
Train_data = Train_data.drop(32180)
Train_data = Train_data.drop(58858)

Train_data['txn_floor'] = Train_data['txn_floor'].fillna(0)
Train_data[Train_data.loc[:,"total_price"] > 1500000000]["txn_floor"]
Train_data = Train_data.drop(37480)

Train_data[Train_data.loc[:,"total_price"] > 2000000000]["building_complete_dt"]
Train_data = Train_data.drop(9491)
Train_data = Train_data.drop(46934)
Train_data = Train_data.drop(50076)

Train_data.sort_values(by = 'parking_price', ascending = False)[:4]
Train_data = Train_data.drop(55472)
Train_data = Train_data.drop(8795)
#Train_data = Train_data.drop(37480)
Train_data = Train_data.drop(51731)
Train_data = Train_data.drop(1351)

Train_data.sort_values(by = 'parking_area', ascending = False)[:2]
Train_data = Train_data.drop(38617)
Train_data = Train_data.drop(41914)
Train_data = Train_data.drop(4339)
Train_data = Train_data.drop(47253)
Train_data = Train_data.drop(38992)
Train_data = Train_data.drop(35611)
Train_data = Train_data.drop(30232)
Train_data = Train_data.drop(55200)

Train_data.sort_values(by = 'land_area', ascending = False)[:10]
Train_data = Train_data.drop(10670)
Train_data = Train_data.drop(41331)
Train_data = Train_data.drop(39191)
Train_data = Train_data.drop(53966)
Train_data = Train_data.drop(7814)
Train_data = Train_data.drop(13332)
Train_data = Train_data.drop(9785)
Train_data = Train_data.drop(14007)
#Train_data = Train_data.drop(50076)
Train_data = Train_data.drop(2086)
Train_data = Train_data.drop(36482)
Train_data = Train_data.drop(11170)
Train_data = Train_data.drop(16838)
Train_data = Train_data.drop(44123)
Train_data = Train_data.drop(59506)
Train_data = Train_data.drop(37526)
Train_data = Train_data.drop(29570)
Train_data = Train_data.drop(23399)

Train_data.sort_values(by = 'building_area', ascending = False)[:20]
#Train_data = Train_data.drop(9491)
#Train_data = Train_data.drop(46934)
Train_data = Train_data.drop(16846)
Train_data = Train_data.drop(16817)
Train_data = Train_data.drop(26767)
Train_data = Train_data.drop(33373)
Train_data = Train_data.drop(18748)
Train_data = Train_data.drop(43513)
Train_data = Train_data.drop(13096)
Train_data = Train_data.drop(20546)
Train_data = Train_data.drop(23605)
Train_data = Train_data.drop(29104)
Train_data = Train_data.drop(21239)
Train_data = Train_data.drop(53345)
Train_data = Train_data.drop(14944)
Train_data = Train_data.drop(44963)
Train_data = Train_data.drop(26014)
Train_data = Train_data.drop(21231)
Train_data = Train_data.drop(36955)

Train_data.sort_values(by = 'I_MIN', ascending = False)[:2]["I_MIN"]
Train_data = Train_data.drop(9119)
Train_data = Train_data.drop(3327)

Train_data.sort_values(by = 'III_MIN', ascending = False)[:3]["III_MIN"]
Train_data = Train_data.drop(52324)
Train_data = Train_data.drop(16859)
Train_data = Train_data.drop(53310)

Train_data.sort_values(by = 'V_MIN', ascending = False)[:2]["V_MIN"]
Train_data = Train_data.drop(9124)
Train_data = Train_data.drop(58248)

Train_data.sort_values(by = 'VII_MIN', ascending = False)[:5]["VII_MIN"]
Train_data = Train_data.drop(25569)
Train_data = Train_data.drop(58812)
Train_data = Train_data.drop(1011)
Train_data = Train_data.drop(34303)
Train_data = Train_data.drop(46392)

Train_data.sort_values(by = 'VIII_MIN', ascending = False)[:5]["VIII_MIN"]
Train_data = Train_data.drop(8686)
Train_data = Train_data.drop(25855)
Train_data = Train_data.drop(2652)

Train_data.sort_values(by = 'XI_MIN', ascending = False)[:5]["XI_MIN"]
Train_data = Train_data.drop(53654)
Train_data = Train_data.drop(15654)
Train_data = Train_data.drop(50764)

Train_data.sort_values(by = 'XII_MIN', ascending = False)[:5]["XII_MIN"]
Train_data = Train_data.drop(32153)
Train_data = Train_data.drop(57831)

#Train_data[Train_data.loc[:,"total_price"] > 21]["XIII_MIN"]
#Train_data = Train_data.drop(58858)

In [277]:
train_Y = np.log1p((Train_data["total_price"]))#np.log1p((Train_data["total_price"] / Train_data["building_area"]))
Train_data = Train_data.drop(['total_price'] , axis=1)

df = pd.concat([Train_data,Test_data])
df.head()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIV_250,XIV_500,XIV_index_500,XIV_1000,XIV_index_1000,XIV_5000,XIV_index_5000,XIV_10000,XIV_index_10000,XIV_MIN
0,8,21,18674,4,3,2,6271,2,NaN,NaN,...,21,58,1,157,1,2483,1,6011,1,34.469803
1,8,7,18800,5,1,2,7885,2,NaN,NaN,...,7,28,1,115,1,15872,1,32221,1,40.073573
2,8,7,19289,4,1,2,6028,2,NaN,NaN,...,27,78,1,212,1,15760,1,32228,1,54.462081
3,8,21,20385,24,0,2,18325,0,NaN,81138.889762,...,2,20,1,125,1,2568,1,7271,1,99.628966
4,1,21,20657,2,4,2,6880,2,NaN,NaN,...,2,18,1,47,1,2587,1,7442,1,124.131236


In [278]:
df["XIII_5000-s2"] = df["XIII_5000"] ** 2
df["XIII_5000-s3"] = df["XIII_5000"] ** 3
df["XIII_5000-Sq"] = np.sqrt(df["XIII_5000"])
df["jobschool_rate-s2"] = df["jobschool_rate"] ** 2
df["jobschool_rate-s3"] = df["jobschool_rate"] ** 3
df["jobschool_rate-Sq"] = np.sqrt(df["jobschool_rate"])
df["bachelor_rate-s2"] = df["bachelor_rate"] ** 2
df["bachelor_rate-s3"] = df["bachelor_rate"] ** 3
df["bachelor_rate-Sq"] = np.sqrt(df["bachelor_rate"])
df["XIII_10000-s2"] = df["XIII_10000"] ** 2
df["XIII_10000-s3"] = df["XIII_10000"] ** 3
df["XIII_10000-Sq"] = np.sqrt(df["XIII_10000"])
df["VII_10000-s2"] = df["VII_10000"] ** 2
df["VII_10000-s3"] = df["VII_10000"] ** 3
df["VII_10000-Sq"] = np.sqrt(df["VII_10000"])
df["IX_10000-s2"] = df["IX_10000"] ** 2
df["IX_10000-s3"] = df["IX_10000"] ** 3
df["IX_10000-Sq"] = np.sqrt(df["IX_10000"])

In [279]:
df['town'] = df['city'] * 1000 + df['town']

In [227]:
# Change the features to categorical
df['building_material'] = df['building_material'].astype(str)
df['city'] = df['city'].astype(str)
df['town'] = df['town'].astype(str)
df['village'] = df['village'].astype(str)
df['building_type'] = df['building_type'].astype(str)
df['building_use'] = df['building_use'].astype(str)
df['parking_way'] = df['parking_way'].astype(str)

In [228]:
categorical_features = df.select_dtypes(include = ["object"]).columns
numerical_features = df.select_dtypes(exclude = ["object"]).columns

In [280]:
print("Numerical features : " + str(len(numerical_features)))
print("Categorical features : " + str(len(categorical_features)))

Numerical features : 244
Categorical features : 7


In [281]:
skewness = df[numerical_features].apply(lambda x: skew(x))
skewness = skewness[abs(skewness) > 1.5]
print(str(skewness.shape[0]) + " skewed numerical features to log transform")
skewed_features = skewness.index
df[skewed_features] = np.log1p(df[skewed_features])

df.head()

131 skewed numerical features to log transform


,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,bachelor_rate-Sq,XIII_10000-s2,XIII_10000-s3,XIII_10000-Sq,VII_10000-s2,VII_10000-s3,VII_10000-Sq,IX_10000-s2,IX_10000-s3,IX_10000-Sq
0,8,21,18674,4,3,2,6271,2,NaN,NaN,...,0.456310,87025,25672375,17.175564,48846121,341385539669,83.600239,3779136,7346640384,44.090815
1,8,7,18800,5,1,2,7885,2,NaN,NaN,...,0.464804,2325625,3546578125,39.051248,346257664,6443162611712,136.411143,19351201,85125933199,66.324958
2,8,7,19289,4,1,2,6028,2,NaN,NaN,...,0.464804,2579236,4142253016,40.074930,429774361,8909652277891,143.982638,22992025,110246759875,69.245939
3,8,21,20385,24,0,2,18325,0,NaN,81138.889762,...,0.456310,97344,30371328,17.663522,51883209,373714754427,84.870490,4137156,8414975304,45.099889
4,1,21,20657,2,4,2,6880,2,NaN,NaN,...,0.456310,7744,681472,9.380832,5978025,14616271125,49.446941,362404,218167208,24.535688


In [285]:
vimm = df.groupby('village').agg({'village_income_median':'mean'})
tvimm = df.groupby('town').agg({'village_income_median':'median'})

vim_isnan = np.isnan(df['village_income_median'])
for i in range(len(df)):
    if vim_isnan.values[i] == True:
        print(i)
        df['village_income_median'].values[i] = vimm.loc[df['village'].values[i], 'village_income_median']
for i in range(len(df)):
    if vim_isnan.values[i] == True:
        df['village_income_median'].values[i] = tvimm.loc[df['town'].values[i], 'village_income_median']

116
438
474
603
749
804
948
968
1031
1093
1141
1144
1226
1275
1301
1429
1430
1580
1773
1791
1915
1989
2034
2055
2070
2140
2212
2273
2305
2341
2432
2475
2545
2600
2623
2733
2752
2758
2957
3006
3055
3078
3080
3119
3211
3229
3231
3243
3318
3410
3414
3508
3554
3566
3690
3695
3728
3747
3957
3974
3977
4062
4100
4102
4123
4166
4215
4220
4251
4298
4315
4351
4400
4485
4488
4567
4635
4677
4696
4909
5082
5118
5273
5311
5346
5378
5471
5527
5563
5573
5577
5668
5689
5695
5809
5836
5840
5876
5958
5989
6050
6105
6171
6335
6358
6427
6431
6529
6563
6589
6758
6771
6799
6958
6965
6994
7081
7127
7210
7270
7290
7294
7319
7341
7391
7414
7450
7451
7529
7556
7593
7656
7682
7687
7712
8067
8069
8073
8090
8109
8127
8138
8142
8148
8220
8225
8232
8258
8285
8318
8347
8391
8424
8538
8563
8567
8608
8691
8720
8738
8773
8779
8849
8870
8871
8977
9050
9051
9053
9081
9136
9141
9151
9242
9453
9485
9493
9496
9517
9548
9729
9738
9744
9788
9799
9831
9929
10005
10033
10035
10060
10095
10104
10146
10154
10199
10377
10429
10485
1

In [286]:
na_check(df)

,Missing Ratio
parking_area,94.861711
parking_price,76.826931
txn_floor,3.773990
village_income_median,0.048623


In [287]:
df_town = df["town"]
df_village = df["village"]
df = df.drop(['town'] , axis=1)
df = df.drop(['village'] , axis=1)
df = pd.get_dummies(df)
df["town"] = df_town
df["village"] = df_village
df.head()

,building_material,city,txn_dt,total_floor,building_type,building_use,building_complete_dt,parking_way,parking_area,parking_price,...,XIII_10000-s3,XIII_10000-Sq,VII_10000-s2,VII_10000-s3,VII_10000-Sq,IX_10000-s2,IX_10000-s3,IX_10000-Sq,town,village
0,8,21,18674,4,3,2,6271,2,NaN,NaN,...,25672375,17.175564,48846121,341385539669,83.600239,3779136,7346640384,44.090815,21334,3132
1,8,7,18800,5,1,2,7885,2,NaN,NaN,...,3546578125,39.051248,346257664,6443162611712,136.411143,19351201,85125933199,66.324958,7180,921
2,8,7,19289,4,1,2,6028,2,NaN,NaN,...,4142253016,40.074930,429774361,8909652277891,143.982638,22992025,110246759875,69.245939,7180,1544
3,8,21,20385,24,0,2,18325,0,NaN,81138.889762,...,30371328,17.663522,51883209,373714754427,84.870490,4137156,8414975304,45.099889,21343,3350
4,1,21,20657,2,4,2,6880,2,NaN,NaN,...,681472,9.380832,5978025,14616271125,49.446941,362404,218167208,24.535688,21102,63


In [288]:
train_num = train_Y.shape[0]

df['txn_floor'] = df['txn_floor'].fillna(0)
df["house_type"] = df['txn_floor'].apply(house_type)
df["lat_diff"] = df['lat'].apply(lat_diff)

In [289]:
df = df.drop(['parking_price'] , axis=1)
df = df.drop(['parking_area'] , axis=1)

In [290]:
temp_df = copy.deepcopy(df)

In [291]:
temp_df = temp_df.fillna(temp_df.mean())

#temp_df["parking_price"] = temp_df["parking_price"].clip(0.1, temp_df["parking_price"].max())
#temp_df["parking_area"] = temp_df["parking_area"].clip(0.1, temp_df["parking_area"].max())

temp_df["txn_dt"] = temp_df["txn_dt"]/100
temp_df["building_complete_dt"] = temp_df["building_complete_dt"]/100

temp_df["village_income_median"] = np.log1p(temp_df["village_income_median"])

temp_df["town_area"] = temp_df["town_area"].clip(1, 150)

temp_df["total_building_area"] = temp_df["building_area"] * temp_df["total_floor"]
temp_df["total_income"] = np.log1p(temp_df["village_income_median"] * temp_df["town_population"])
temp_df["area_percent"] = temp_df["land_area"] / temp_df["town_area"]
#temp_df["per_parking_price"] = np.log1p(np.expm1(temp_df["parking_price"]) / temp_df["parking_area"])

temp_df["area_diff"] = temp_df["land_area"] - temp_df["building_area"]
temp_df['area_type'] = temp_df['area_diff'].apply(area_type)
#temp_df = temp_df.drop(['area_diff'] , axis=1)

temp_df = temp_df.fillna(temp_df.mean())

train_num = train_Y.shape[0]
train_X = temp_df[:train_num]
test_X = temp_df[train_num:]

In [292]:
temp_df = temp_df.drop(['doc_rate', 'master_rate', "bachelor_rate", 'jobschool_rate', 'highschool_rate', "elementary_rate"] , axis=1)
#temp_df = temp_df.drop(['XIII_5000'] , axis=1)

In [293]:
temp_df.reset_index(drop=True, inplace=True)
train_Y.reset_index(drop=True, inplace=True)

In [27]:
Data = pd.concat([temp_df[:train_num], train_Y], axis=1)
test_Data = temp_df[train_num:]

temp_df["txn_floor"] = temp_df["txn_floor"].astype('int')

cols = ["city", "building_type", "building_use", "total_floor", "txn_floor", "lon", "lat"]

for c in cols:
    
    mean_df = Data.groupby([c])['total_price'].mean().reset_index()
    mean_df.columns = [c, f'{c}_mean']
    
    Data = pd.merge(Data, mean_df, on= c, how='left')
    Data = Data.drop([c] , axis=1)
    
    test_Data = pd.merge(test_Data, mean_df, on= c, how='left')   
    test_Data = test_Data.drop([c] , axis=1)
    
Data = Data.drop(['total_price'] , axis=1)

temp_df = pd.concat([Data,test_Data])
temp_df.head()

,building_material,txn_dt,building_complete_dt,parking_way,land_area,building_area,village_income_median,town_population,town_area,town_population_density,...,area_percent,area_diff,area_type,city_mean,building_type_mean,building_use_mean,total_floor_mean,txn_floor_mean,lon_mean,lat_mean
0,8,186.74,62.71,2,2.952013,1.485727,6.350886,24219.0,1.000000,17103.0,...,2.952013,1.466287,1,14.760251,15.616688,15.494418,15.601214,15.427320,14.625019,14.841126
1,8,188.00,78.85,2,2.516666,1.617666,6.311735,551480.0,3.183758,23835.0,...,0.790470,0.899000,1,15.731945,15.368033,15.494418,15.396904,15.357832,15.483269,15.907105
2,8,192.89,60.28,2,3.110257,1.884685,6.439350,551480.0,3.183758,23835.0,...,0.976914,1.225572,1,15.731945,15.368033,15.494418,15.601214,15.781742,15.681289,16.098430
3,8,203.85,183.25,0,2.516666,2.678486,6.857514,139259.0,2.756986,9440.0,...,0.912832,-0.161820,0,14.760251,15.835580,15.494418,16.127585,15.832799,14.723749,15.345815
4,1,206.57,68.80,2,4.132066,1.738378,6.285998,112397.0,4.277222,1582.0,...,0.966063,2.393688,1,14.760251,15.420499,15.494418,14.951294,15.420499,14.783370,14.291252


In [294]:
temp_df2 = copy.deepcopy(temp_df)

In [295]:
temp_df2 = temp_df2.fillna(temp_df2.mean())

train_num = train_Y.shape[0]
train_X = temp_df2[:train_num]
test_X = temp_df2[train_num:]

In [296]:
temp_train = copy.deepcopy(train_X)
temp_train["total_price"] = train_Y

corr = temp_train.corr()["total_price"]
high_feature = corr[abs(corr) >= 0.001]

In [297]:
high_feature = list(high_feature.index)

remove = []

for f in high_feature:
    if ("index" in f):
        remove.append(f)
    #elif ("_" in f) and (int(f.split("_")[1]) < 5000):
    #    remove.append(f)
    if ("MIN" in f):
        remove.append(f)
        
for f in remove:
    high_feature.remove(f)

In [298]:
high_feature.remove("total_price")
train_X = train_X[high_feature]
test_X = test_X[high_feature]

In [166]:
est = RandomForestRegressor()
est.fit(train_X, train_Y)
feats_est = pd.Series(data=est.feature_importances_, index=train_X.columns)
feats_est = feats_est.sort_values(ascending=False)
feats_est

/home/rex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


VI_10000                 1.673305e-01
XIII_10000-s3            1.141072e-01
XIII_10000               1.140094e-01
XIII_10000-s2            6.069009e-02
XIII_10000-Sq            5.762052e-02
VII_10000-Sq             5.545226e-02
building_complete_dt     4.995715e-02
jobschool_rate-s3        3.034653e-02
bachelor_rate-s3         2.144204e-02
village_income_median    2.006354e-02
area_diff                1.994472e-02
txn_dt                   1.956669e-02
bachelor_rate-s2         1.376287e-02
marriage_rate            1.255173e-02
txn_floor                1.219742e-02
bachelor_rate-Sq         1.062624e-02
building_area            1.006020e-02
land_area                9.259776e-03
jobschool_rate-Sq        9.210384e-03
divorce_rate             9.037108e-03
V_5000                   8.960273e-03
jobschool_rate-s2        7.595743e-03
total_floor              6.753331e-03
X_5000                   6.293962e-03
VII_1000                 6.086915e-03
XIII_5000-Sq             5.465663e-03
V_10000     

In [299]:
corr[high_feature].sort_values(ascending=False)

building_area            0.561034
jobschool_rate-s3        0.505768
XIII_5000                0.501536
jobschool_rate-s2        0.501118
XIII_10000-Sq            0.496011
bachelor_rate-Sq         0.492984
VII_10000-Sq             0.492443
XIII_5000-Sq             0.490425
XIII_10000               0.490170
jobschool_rate-Sq        0.489572
VII_10000                0.485678
bachelor_rate-s2         0.485195
IX_10000-Sq              0.481756
IX_10000                 0.479959
V_10000                  0.479498
bachelor_rate-s3         0.478700
VIII_10000               0.470670
III_10000                0.463622
VII_10000-s2             0.460694
XI_10000                 0.460193
X_10000                  0.460166
XIII_10000-s2            0.459805
VI_10000                 0.457410
II_10000                 0.455389
IX_10000-s2              0.454682
XII_10000                0.451279
village_income_median    0.443107
VII_10000-s3             0.436371
IV_10000                 0.435527
XIII_5000-s2  

In [300]:
# 切分訓練集/測試集
x_train, x_test, y_train, y_test = train_test_split(train_X, train_Y, test_size=0.25, random_state=4)

estimator_ = RandomForestRegressor()
estimator_.fit(x_train, y_train)

print(cross_val_score(estimator_, x_train, y_train, cv=5).mean())

y_pred = estimator_.predict(x_test)
print("Mean squared error: %.4f"
      % mean_squared_error(y_test, y_pred))

/home/rex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9430515415792404
Mean squared error: 0.0742


In [301]:
estimator_ = RandomForestRegressor()
estimator_.fit(train_X, train_Y)

predict = estimator_.predict(test_X)

test_price = np.expm1(predict)# * (test_X['building_area'].values)

submit = pd.DataFrame(ids)
submit['total_price'] = test_price
submit.to_csv("2019-07-09.csv",index=False)

/home/rex/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [197]:
test_price

array([6051599.96984175, 1114836.18197655, 1994430.20112747, ...,
        314851.49687368, 1278756.10853138,  710310.16706749])